In [1]:
import matplotlib.pyplot as plt
from matplotlib  import rc
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import plotly.express as px
from plotly import graph_objs as go
import plotly.colors as colors


## Load bank-tract + census data

We want to plot BOW variables. BOW doesn't have observations in tracts without loans, so we first add the missing tracts and set loan volume equal to 0. 

In [2]:
bank_tract = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

# Create a copy of the original dataframe
bank_tract_new = bank_tract.copy()

# Find the census tracts that have AllOtherBanks but not BankofWest
missing_tracts = bank_tract_new[(bank_tract_new['which_bank'] == 'All Other Banks') & ~(bank_tract_new['census_tract'].isin(bank_tract_new[bank_tract_new['which_bank'] == 'Bank of West']['census_tract']))].copy()

# Update which_bank column to BankofWest
missing_tracts['which_bank'] = 'Bank of West'

# Set bank specific variables to 0
start_col = bank_tract.columns.get_loc('sum_approved_loans')
bank_specific_vars = bank_tract.columns[start_col:]
missing_tracts[bank_specific_vars] = np.NaN
missing_tracts['log_num_apps'] = 0

# Append the missing rows to the new dataframe
bank_tract_new = pd.concat([bank_tract_new, missing_tracts], ignore_index=True)

# Verify that there are now equal numbers of rows for each which_bank value
bank_tract_new["which_bank"].value_counts()

Bank of West       9340
All Other Banks    9337
Name: which_bank, dtype: int64

In [3]:
bank_tract = bank_tract_new
#denial_rate, credit_spread, np.log(# loans), np.log($ loans), avg LTV 
bank_tract ["white_rate"]        = (bank_tract ["WhitePop"]) / bank_tract ["Tot.Pop"] * 100
bank_tract ['majority-minority'] = bank_tract ["white_rate"] < 50
bank_tract ['below_p10_income']  = bank_tract ["Med.HousehIncome"] < bank_tract["Med.HousehIncome"].quantile(0.1)
bank_tract ['high_hispanic']     = (bank_tract ['HispanicLatinoPop'] / bank_tract['Tot.Pop']) > 0.25
bank_tract ['high_black']        = (bank_tract ['BlackPop'] / bank_tract['Tot.Pop']) > 0.25
bank_tract ['high_asian']        = (bank_tract ['AsianPop'] / bank_tract['Tot.Pop']) > 0.25
bank_tract ['high_white']        = (bank_tract ['WhitePop'] / bank_tract['Tot.Pop']) > 0.25

bank_tract ['log(numloans)']     = np.log(bank_tract['num_approved_loans'])
bank_tract ['log(dol_loans)']    = np.log(bank_tract['avg_approved_loan_size'])

## Reduce to focal counties 

This makes the map files smaller. 


In [4]:
bank_tract = bank_tract.query('(state == 4 & county in [13,19]) | (state == 6 & county in [37,75, 81, 1,85]) | (state == 18 & county in [97])')


## Reduce to tract level data and plot BOW only 


In [5]:
tract_level = bank_tract.query('which_bank == "Bank of West"').copy()

### Get the shapefiles

In [6]:
# Get the Arizona tract shapefile and convert to UTM Zone 17N coordinate system
shape_az = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_04_tract.zip") #.to_crs(epsg=32617)
shape_ca = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_06_tract.zip") #.to_crs(epsg=32617)
shape_all = pd.concat([shape_az, shape_ca], ignore_index=True)
shape_all['census_tract'] = shape_all['GEOID'].astype(np.int64)

In [7]:
tract_with_shape = shape_all.merge(tract_level, how = 'inner', on = ['census_tract'],
                                   indicator = True, validate= '1:m') 

### Plot

- color for the shapes
- red dots for areas of concern (defined by...)

In [8]:
concerns = tract_with_shape.query('`majority-minority`').copy()
# we need this for the "area of concern" red marks
concerns['lon'] = concerns['INTPTLON'].astype(float)
concerns['lat'] = concerns['INTPTLAT'].astype(float)

In [9]:
listofshading   = ['denial_rate', 'mean_approved_rate_spread', 'log(numloans)', 'log(dol_loans)', 'mean_LTV', 'white_rate', 
                  'majority-minority','below_p10_income', 'high_hispanic', 'high_black', 'high_asian', 'high_white'] 

for v in listofshading:

    fig = px.choropleth_mapbox(tract_with_shape.set_index('GEOID'),
                               geojson=tract_with_shape.geometry,
                               locations=tract_with_shape.index,
                               opacity=.7,
                               color=v, color_continuous_scale=px.colors.sequential.Greens,
                               center={"lat": 33.44, "lon": -112.074036},
                               mapbox_style="open-street-map",
                               zoom=5)

    fig.add_scattermapbox(
        lat = concerns['lat'],
        lon = concerns['lon'],
        mode = 'markers',
        marker_size=12,
        marker_color='red'
    )

    fig.update_layout(
        title=v,
        autosize=False,
        width=1000,
        height=1000,
        margin={"r":0,"t":0,"l":0,"b":0}
    )
    fig.write_html(f"../images/BOW_{v}.html")

## Shading now shows difference between BOW can competitors 


In [10]:
bank_tract = bank_tract.query('(state == 4 & county in [13,19]) | (state == 6 & county in [37,75, 81, 1,85])') # | (state == 18 & county in [97])
# bank_tract["which_bank"].value_counts()

grouped = bank_tract.groupby(['census_tract', 'which_bank']) 
denial_rates = grouped['denial_rate'].mean()

bank_of_west = denial_rates.loc[(slice(None), 'Bank of West')]
all_other_banks = denial_rates.loc[(slice(None), 'All Other Banks')]
percent_difference = bank_of_west - all_other_banks

In [11]:
df_percent_difference = pd.DataFrame({'census_tract': percent_difference.index.get_level_values(0), 'denial_rate_percent_difference': percent_difference.values})

bank_tract = pd.merge(bank_tract, df_percent_difference, on='census_tract', how='left')

In [12]:
shape_all = pd.concat([shape_az, shape_ca], ignore_index=True)
shape_all['census_tract'] = shape_all['GEOID'].astype(np.int64)

tract_with_shape = shape_all.merge(bank_tract, how = 'inner', on = ['census_tract'],
                                   indicator = True, validate= '1:m') 

In [13]:
concerns = tract_with_shape.query('`majority-minority`').copy()
# we need this for the "area of concern" red marks
concerns['lon'] = concerns['INTPTLON'].astype(float)
concerns['lat'] = concerns['INTPTLAT'].astype(float)

listofshading = ['denial_rate_percent_difference']

In [14]:
colorscale = colors.sequential.Greens[::-1] + colors.sequential.Reds

for v in listofshading:

    fig = px.choropleth_mapbox(tract_with_shape.set_index('GEOID'),
                               geojson=tract_with_shape.geometry,
                               locations=tract_with_shape.index,
                               opacity=.7,
                               color=v, color_continuous_scale=colorscale,
                               center={"lat": 33.44, "lon": -112.074036},
                               mapbox_style="open-street-map",
                               zoom=5)

    fig.add_scattermapbox(
        lat = concerns['lat'],
        lon = concerns['lon'],
        mode = 'markers',
        marker_size=12,
        marker_color='red'
    )

    fig.update_layout(
        title=v,
        autosize=False,
        width=1000,
        height=1000,
        margin={"r":0,"t":0,"l":0,"b":0}
    )
    fig.write_html(f"../images/{v}.html")